In [3]:
library(repr)
library(ggplot2)
library(gutenbergr)
options(jupyter.plot_mimetypes = 'image/png')

# Parse the Gutemberg data base 

+ use the [gutemberger library](https://cran.r-project.org/web/packages/gutenbergr/README.html)
+ use the `grepl` command to filter the requiered data set

In [12]:
french_books = gutenberg_works(only_text=1, only_languages=1, languages="fr")
maupassant_books = french_works[grepl("Maupassant", french_works$author), ]
maupassant_books[c(1,2,3),]

gutenberg_id,title,author,gutenberg_author_id,language,gutenberg_bookshelf,rights,has_text
10746,Boule de Suif,"Maupassant, Guy de",306,fr,FR Nouvelles,Public domain in the USA.,TRUE
10775,Le Horla,"Maupassant, Guy de",306,fr,Gothic Fiction/FR Nouvelles,Public domain in the USA.,TRUE
11131,Pierre et Jean,"Maupassant, Guy de",306,fr,FR Littérature,Public domain in the USA.,TRUE


# Download the books
The result is a two column `tbl_df` dataframe from the `dplyr` library. It has one row per line of the text.
This dataframes are only a wrapper of the R dataframes to avoid long prints

In [17]:
maupassant_downloaded_books <- gutenberg_download(maupassant_books$gutenberg_id)

In [19]:
write.csv(maupassant_downloaded_books,"maupassant.csv")